In [1]:
 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
print(os.listdir("../input"))



['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
train_df=pd.read_csv('../input/train.csv')
test_df=pd.read_csv('../input/test.csv')

In [3]:

train_df['labels_text'] = '__label__' + train_df.target.map(str)
train_df.labels_text = train_df.labels_text.str.cat(train_df.question_text, sep=' ')

# Write training data to a file as required by fasttext
training_file = open('train.txt','w')
training_file.writelines(train_df.labels_text + '\n')
training_file.close()


print(os.listdir("./"))

['train.txt', '__notebook_source__.ipynb', '.ipynb_checkpoints']


In [ ]:
import fastText as ft
import pandas as pd
import numpy as np 
import re
import fastText as ft
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from collections import Counter
# Function to do K-fold CV across different fasttext parameter values
def tune(Y, X, YX, k, lr, wordNgrams, epoch):
    # Record results
    results = []
    for lr_val in lr:
        for wordNgrams_val in wordNgrams:
            for epoch_val in epoch:  
                # K-fold CV
                kf = KFold(n_splits=k, shuffle=True)
                fold_results = []
                # For each fold
                for train_index, test_index in kf.split(X):
                    # Write the training data for this CV fold
                    training_file = open('train_cv.txt','w')
                    training_file.writelines(YX[train_index] + '\n')
                    training_file.close()
                    # Fit model for this set of parameter values
                    model = ft.FastText.train_supervised('train_cv.txt',
                                          lr=lr_val,
                                          wordNgrams=wordNgrams_val,
                                          epoch=epoch_val)
                    # Predict the holdout sample
                    pred = model.predict(X[test_index].tolist())
                    pred = pd.Series(pred[0]).apply(lambda x: re.sub('__label__', '', x[0]))
                    # Compute accuracy for this CV fold
                    fold_results.append(accuracy_score(Y[test_index], pred.values))
                # Compute mean accuracy across 10 folds
                mean_acc = pd.Series(fold_results).mean()
                print([lr_val, wordNgrams_val, epoch_val, mean_acc])
    # Add current parameter values and mean accuracy to results table
    results.append([lr_val, wordNgrams_val, epoch_val, mean_acc])         
    # Return as a DataFrame 
    results = pd.DataFrame(results)
    results.columns = ['lr','wordNgrams','epoch','mean_acc']
    return(results)
# Tune parameters using 10-fold CV
#results = tune(Y = train_df.target.map(str),
#               X = train_df.question_text,
#               YX = train_df.labels_text,
#               k = 10, 
#               lr = [0.05, 0.1, 0.2],
#               wordNgrams = [1,2,3],
#               epoch = [15,17,20]
#              )
# Train final classifiers
classifier1 = ft.FastText.train_supervised('train.txt', lr=0.1, wordNgrams=1, epoch=500)
classifier2 = ft.FastText.train_supervised('train.txt', lr=0.1, wordNgrams=2, epoch=500)
classifier3 = ft.FastText.train_supervised('train.txt', lr=0.1, wordNgrams=3, epoch=500)
print("DONE")

In [ ]:
mlist=test_df["question_text"].map(lambda s: s.strip()).tolist()
#print(test_df["question_text"])

mlist

In [ ]:

predictions1 = classifier1.predict(mlist)
predictions2 = classifier2.predict(mlist)
predictions3 = classifier3.predict(mlist)

In [ ]:
#print(predictions1)
#print(predictions2)
#print(predictions3)

In [ ]:
# Combine predictions
majority_vote = np.array([])
for i in range(len(predictions1[0])):
    majority_vote = np.append(majority_vote, Counter([predictions1[0][i][0],
                                                   predictions2[0][i][0],
                                                   predictions3[0][i][0]]).most_common(1)[0][0])


In [ ]:

import os
import glob

import os, shutil
folder = './'
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

print(os.listdir("./"))    

In [ ]:


submit = pd.DataFrame({'qid': test_df["qid"].values, 
                       'prediction': pd.Series(majority_vote)})
submit.prediction = submit.prediction.apply(lambda x: re.sub('__label__', '', x))
submit.to_csv('submission.csv', index=False)
submit